# E-commerce Sales - Data Cleaning

**Objective:** Clean and prepare the raw data for analysis.

**Input:** Raw transaction data  
**Output:** Cleaned dataset ready for analysis

**Author:** [Your Name]  
**Date:** [Date]

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print('✅ Libraries loaded')

In [ ]:
# Load raw data
try:
    df = pd.read_excel('../data/raw/online_retail_II.xlsx', sheet_name='Year 2010-2011')
    print(f'✅ Loaded full dataset: {len(df):,} rows')
except FileNotFoundError:
    df = pd.read_csv('../data/sample/sample_data.csv')
    print(f'⚠️ Using sample data: {len(df):,} rows')

original_rows = len(df)
print(f'\nStarting with {original_rows:,} rows')

## 2. Standardize Column Names

In [ ]:
# Standardize column names
print('📋 Standardizing column names...')
print(f'   Before: {list(df.columns)}')

# Rename for consistency
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Common renames
rename_map = {
    'Customer_ID': 'Customer ID',
    'CustomerID': 'Customer ID',
    'InvoiceNo': 'Invoice',
    'UnitPrice': 'Price'
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

print(f'   After: {list(df.columns)}')

## 3. Handle Missing Values

In [ ]:
print('🔧 Handling missing values...')

# Check missing
missing_before = df.isnull().sum()
print(f'\n   Missing values before:')
print(missing_before[missing_before > 0])

# Customer ID: Keep rows with missing (guest checkouts) but flag them
df['IsGuest'] = df['Customer ID'].isnull().astype(int)
print(f'\n   Guest checkouts flagged: {df["IsGuest"].sum():,}')

# Description: Fill with 'Unknown' or drop
if 'Description' in df.columns:
    df['Description'] = df['Description'].fillna('Unknown')

print('\n   ✅ Missing values handled')

## 4. Remove Cancellations and Returns

In [ ]:
print('🔧 Removing cancellations and returns...')

# Flag cancellations (Invoice starts with 'C')
df['IsCancellation'] = df['Invoice'].astype(str).str.startswith('C')
cancellations = df['IsCancellation'].sum()
print(f'   Cancellations found: {cancellations:,}')

# Remove cancellations for analysis
df_clean = df[~df['IsCancellation']].copy()
print(f'   Rows after removing cancellations: {len(df_clean):,}')

## 5. Handle Invalid Quantities and Prices

In [ ]:
print('🔧 Handling invalid quantities and prices...')

# Remove negative quantities (returns already removed with cancellations)
neg_qty = (df_clean['Quantity'] <= 0).sum()
print(f'   Negative/zero quantities: {neg_qty:,}')
df_clean = df_clean[df_clean['Quantity'] > 0]

# Remove zero/negative prices
invalid_price = (df_clean['Price'] <= 0).sum()
print(f'   Invalid prices: {invalid_price:,}')
df_clean = df_clean[df_clean['Price'] > 0]

print(f'   Rows remaining: {len(df_clean):,}')

## 6. Remove Outliers

In [ ]:
print('🔧 Handling outliers...')

# Calculate revenue
df_clean['Revenue'] = df_clean['Quantity'] * df_clean['Price']

# Remove extreme outliers (>99.9th percentile)
qty_threshold = df_clean['Quantity'].quantile(0.999)
price_threshold = df_clean['Price'].quantile(0.999)
revenue_threshold = df_clean['Revenue'].quantile(0.999)

print(f'   Quantity 99.9th percentile: {qty_threshold:.0f}')
print(f'   Price 99.9th percentile: £{price_threshold:.2f}')
print(f'   Revenue 99.9th percentile: £{revenue_threshold:.2f}')

# Filter extreme outliers
before_outliers = len(df_clean)
df_clean = df_clean[
    (df_clean['Quantity'] <= qty_threshold) &
    (df_clean['Price'] <= price_threshold) &
    (df_clean['Revenue'] <= revenue_threshold)
]
outliers_removed = before_outliers - len(df_clean)
print(f'   Outliers removed: {outliers_removed:,}')

## 7. Add Derived Features

In [ ]:
print('🔧 Adding derived features...')

# Ensure datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Date features
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour
df_clean['IsWeekend'] = df_clean['DayOfWeek'].isin([5, 6]).astype(int)

print('   Added: Year, Month, DayOfWeek, Hour, IsWeekend')

# Revenue already calculated above
print('   Added: Revenue (Quantity × Price)')

## 8. Validate Final Dataset

In [ ]:
print('✅ Validation Checks')
print('=' * 50)

# No missing values in key columns
key_cols = ['Invoice', 'StockCode', 'Quantity', 'Price', 'InvoiceDate', 'Country', 'Revenue']
missing_key = df_clean[key_cols].isnull().sum().sum()
print(f'Missing in key columns: {missing_key} ✅' if missing_key == 0 else f'Missing in key columns: {missing_key} ❌')

# All quantities positive
neg_qty = (df_clean['Quantity'] <= 0).sum()
print(f'Negative quantities: {neg_qty} ✅' if neg_qty == 0 else f'Negative quantities: {neg_qty} ❌')

# All prices positive
neg_price = (df_clean['Price'] <= 0).sum()
print(f'Negative prices: {neg_price} ✅' if neg_price == 0 else f'Negative prices: {neg_price} ❌')

# All revenues positive
neg_rev = (df_clean['Revenue'] <= 0).sum()
print(f'Negative revenue: {neg_rev} ✅' if neg_rev == 0 else f'Negative revenue: {neg_rev} ❌')

## 9. Save Cleaned Data

In [ ]:
print('💾 Saving cleaned data...')

# Create output directory
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Save
df_clean.to_csv(output_dir / 'cleaned_transactions.csv', index=False)
print(f'   Saved to: {output_dir}/cleaned_transactions.csv')
print(f'   Final rows: {len(df_clean):,}')

## 10. Cleaning Summary

In [ ]:
print('=' * 60)
print('📊 CLEANING SUMMARY')
print('=' * 60)

rows_removed = original_rows - len(df_clean)
pct_removed = rows_removed / original_rows * 100

print(f"""
ROWS:
• Original: {original_rows:,}
• Final: {len(df_clean):,}
• Removed: {rows_removed:,} ({pct_removed:.1f}%)

CLEANING STEPS:
1. Standardized column names
2. Flagged guest checkouts (missing Customer ID)
3. Removed cancellations (Invoice starting with 'C')
4. Removed negative/zero quantities
5. Removed invalid prices (≤0)
6. Removed extreme outliers (>99.9th percentile)
7. Added derived features (Year, Month, DayOfWeek, Hour, Revenue)

FINAL DATASET:
• Transactions: {len(df_clean):,}
• Customers: {df_clean['Customer ID'].nunique():,}
• Products: {df_clean['StockCode'].nunique():,}
• Countries: {df_clean['Country'].nunique()}
• Total Revenue: £{df_clean['Revenue'].sum():,.2f}

✅ Ready for analysis in 03_analysis.ipynb
""")